In [27]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import pymongo
from pprint import pprint

# --- Connect to SQLite ---
conn = sqlite3.connect("../../sql/openfda_base_updated.db")
cursor = conn.cursor()
#11090837

In [4]:
# count entries in drug table where safetyreportid is 11090837
cursor.execute("SELECT COUNT(*) FROM drug WHERE safetyreportid = 11090837")
count = cursor.fetchone()[0]
print(f"Number of entries in drug table with safetyreportid 11090837: {count}")

Number of entries in drug table with safetyreportid 11090837: 561


In [6]:
### count reactions for SELECT COUNT(*) FROM drug WHERE safetyreportid = 11090837

cursor.execute("SELECT COUNT(*) FROM reaction WHERE safetyreportid = 11090837")
count = cursor.fetchone()[0]
print(f"Number of entries in drug table with safetyreportid 11090837: {count}")

Number of entries in drug table with safetyreportid 11090837: 8


In [8]:
### print all reactions for SELECT COUNT(*) FROM drug WHERE safetyreportid = 11090837
cursor.execute("SELECT * FROM reaction WHERE safetyreportid = 11090837")
reactions = cursor.fetchall()
for reaction in reactions:
    print(reaction[1:])

(11090837, 'Coma', 26.1, 1)
(11090837, 'Gastrooesophageal reflux disease', 26.1, 1)
(11090837, 'Pneumonia aspiration', 26.1, 1)
(11090837, 'Somnolence', 26.1, 1)
(11090837, 'Incorrect route of product administration', 26.1, 1)
(11090837, 'Off label use', 26.1, 1)
(11090837, 'Incorrect dose administered by product', 26.1, 1)
(11090837, 'Underdose', 26.1, 1)


In [28]:
cursor.execute("SELECT * FROM patient_optional WHERE safetyreportid = 11090837")
patient_optional = cursor.fetchall()
for patient in patient_optional:
    print(patient)

(11090837, 5, 64, 801, 2, 69.0)


In [16]:
cursor.execute("SELECT COUNT(*) FROM report_duplicate WHERE safetyreportid = 11090837")
count = cursor.fetchone()[0]
print(f"Number of entries in report_duplicate table with safetyreportid 11090837: {count}")

Number of entries in report_duplicate table with safetyreportid 11090837: 105


In [20]:
# count entries in drug table where safetyreportid is 11090837
cursor.execute("SELECT medicinalproduct FROM drug WHERE safetyreportid = 11090837")
count = cursor.fetchall()

unique_medicinal_products = set()
for row in count:
    unique_medicinal_products.add(row[0])
print(count)

[('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('TIZANIDINE HYDROCHLORIDE',), ('TIZANIDINE HYDROCHLORIDE',), ('FUROSEMIDE',), ('PREDNISONE',), ('WARFARIN',), ('BIFIDOBACTERIUM LONGUM\\LACTOBACILLUS ACIDOPHILUS',), ('POTASSIUM CHLORIDE',), ('POTASSIUM CHLORIDE',), ('POTASSIUM CHLORIDE',), ('POTASSIUM CHLORIDE',), ('ATIVAN',), ('VITAMIN B',), ('BACLOFEN',), ('CELLCEPT',), ('CYMBALTA',), ('CYMBAL

In [22]:
print(len(unique_medicinal_products))
print(unique_medicinal_products)

106
{'DULOXETINE', 'SULFAMETHOXAZOLE', 'DULOXETINE HYDROCHLORIDE', 'HYDROMORPHONE', 'BACLOFEN', 'CELLCEPT', 'RIBOFLAVIN', 'BROMHEXINE HYDROCHLORIDE', 'HUMAN IMMUNOGLOBULIN G', 'OMEGA-3 FATTY ACIDS', 'RANITIDINE', 'HORSE CHESTNUT', 'THIAMINE HYDROCHLORIDE', 'SULFADIAZINE', 'RETINOL', 'PYRIDOXINE HYDROCHLORIDE', 'TIZANIDINE', 'WARFARIN', 'TRIMETHOPRIM', 'LORATADINE', 'GLICLAZIDE', 'CYMBALTA', 'FUROSEMIDE', 'MOMETASONE FUROATE', 'DOCONEXENT', 'LORAZEPAM', 'POTASSIUM CHLORIDE', 'HYDROCHLOROTHIAZIDE', 'ALKA-SELTZER', 'VITAMIN B', 'DEXPANTHENOL', 'SULFAMETHOXAZOLE\\TRIMETHOPRIM', 'CHOLECALCIFEROL', 'FISH OIL', 'BIFIDOBACTERIUM BIFIDUM', 'ALEVE', 'NORTRIPTYLINE HYDROCHLORIDE', 'ERGOCALCIFEROL', 'TRIAMTERENE', 'BIFIDOBACTERIUM ANIMALIS LACTIS', 'ORYCTOLAGUS CUNICULUS SKIN', 'ALKA-SELTZER HEARTBURN RELIEFCHEWS', 'FRUCTOSE', 'LYRICA', 'VITAMIN B12', 'NIACINAMIDE\\PYRIDOXINE HYDROCHLORIDE\\RIBOFLAVIN\\THIAMINE HYDROCHLORIDE', 'THIAMINE', 'MYCOPHENOLATE MOFETIL', 'CALCIUM CARBONATE', 'SULFALENE', 

In [30]:
# get the drug ids for the drugs form safetyreportid 11090837
cursor.execute("SELECT id FROM drug WHERE safetyreportid = 11090837")
drug_ids = cursor.fetchall()
ids = []
for drug_id in drug_ids:
    ids.append(drug_id[0])
# for drug_id in drug_ids:
    # print(drug_id[0])

In [35]:
print(ids[:10])

[116519, 116520, 116521, 116522, 116523, 116524, 116525, 116526, 116527, 116528]


In [33]:
# Check how many times each drug id from ids is not present in the drug_activesubstance table
missing_active_substances = []

for drug_id in ids:
    cursor.execute(f"SELECT COUNT(*) FROM drug_activesubstance WHERE drug_id = {drug_id}")
    count = cursor.fetchone()[0]
    if count == 0:
        missing_active_substances.append(drug_id)

print(f"Number of drugs without active substances: {len(missing_active_substances)}")
print(f"Percentage of drugs without active substances: {len(missing_active_substances) / len(ids) * 100:.2f}%")

# Get the medicinal products for these missing active substances
if missing_active_substances:
    # Get a sample of up to 10 missing drugs to display
    sample_size = min(10, len(missing_active_substances))
    sample_ids = missing_active_substances[:sample_size]
    
    placeholders = ', '.join('?' for _ in sample_ids)
    query = f"SELECT id, medicinalproduct FROM drug WHERE id IN ({placeholders})"
    
    cursor.execute(query, sample_ids)
    sample_drugs = cursor.fetchall()
    
    print("\nSample of drugs without active substances:")
    for drug_id, medicinal_product in sample_drugs:
        print(f"ID: {drug_id}, Medicinal Product: {medicinal_product}")

Number of drugs without active substances: 0
Percentage of drugs without active substances: 0.00%


In [34]:
missing_fda = []

for drug_id in ids:
    cursor.execute(f"SELECT COUNT(*) FROM drug_openfda WHERE drug_id = {drug_id}")
    count = cursor.fetchone()[0]
    if count == 0:
        missing_fda.append(drug_id)

print(f"Number of drugs without active substances: {len(missing_fda)}")
print(f"Percentage of drugs without active substances: {len(missing_fda) / len(ids) * 100:.2f}%")

# Get the medicinal products for these missing active substances
if missing_fda:
    # Get a sample of up to 10 missing drugs to display
    sample_size = min(10, len(missing_fda))
    sample_ids = missing_fda[:sample_size]
    
    placeholders = ', '.join('?' for _ in sample_ids)
    query = f"SELECT id, medicinalproduct FROM drug WHERE id IN ({placeholders})"
    
    cursor.execute(query, sample_ids)
    sample_drugs = cursor.fetchall()
    
    print("\nSample of drugs without active substances:")
    for drug_id, medicinal_product in sample_drugs:
        print(f"ID: {drug_id}, Medicinal Product: {medicinal_product}")

Number of drugs without active substances: 0
Percentage of drugs without active substances: 0.00%
